# Simple analysis for contrasts musicians/non-musicians.

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch

### Import dataset.

In [ ]:
colors = ['#1f77b4', '#ff7f0e']

In [ ]:
dname = '/media/jacques/DATA1/2019_MusicPred/experimentContinuousMatrix/data/behavior/'

# Experimental
MUSICIANS = np.load(dname + 'MUSICIANS.npy')
CHOICES = np.load(dname + 'CHOICES.npy')
DEC = np.load(dname + 'DEC.npy')
SEQS = np.load(dname + 'SEQS.npy')
PERF = np.load(dname + 'PERF.npy')
RT = np.load(dname + 'RT.npy')
SEQ_ID = np.load(dname + 'SEQ_ID.npy')
sujs = np.load(dname + 'sujs.npy', allow_pickle=True)

# Training
CHOICES_TRAINING = np.load(dname + 'CHOICES_TRAINING.npy')
SEQS_TRAINING = np.load(dname + 'SEQS_TRAINING.npy')
PERF_TRAINING = np.load(dname + 'PERF_TRAINING.npy')
RT_TRAINING = np.load(dname + 'RT_TRAINING.npy')

SEQS.shape

In [ ]:
n_sujs, n_seqs, n_tones = SEQS.shape
n_dec = RT.shape[-1]
N_VOC = np.unique(SEQS[np.logical_not(np.isnan(SEQS))]).size

# Performances musicians/non-musicians in the training.

In [ ]:
print(np.mean(np.nanmean(PERF_TRAINING, axis=1)))

In [ ]:
print(np.std(np.nanmean(PERF_TRAINING, axis=1))/np.sqrt(n_sujs))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 5))

# Suj
for j, musi in enumerate(np.unique(MUSICIANS)):
    M = np.nanmean(PERF_TRAINING[MUSICIANS[:, 0, 0] == musi], 1)
    m = np.nanmean(M, 0)
    se = np.nanstd(M, 0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
    label = 'Musicians' if musi == True else 'Non-musicians'
    ax.errorbar(j, m, yerr=se, marker='D', linewidth=2, markersize=7, 
                label=label, color=colors[j])

for s, suj in enumerate(sujs):
    ax.plot(np.random.normal(0, 0.01) + MUSICIANS[s, 0, 0], np.nanmean(PERF_TRAINING[s], 0), marker='o', 
            linewidth=1, color=colors[MUSICIANS[s, 0, 0]], linestyle='--', markersize=4, alpha=0.3)

ax.set_xticks(np.arange(2))
ax.set_xticklabels(['Non-musicians', 'Musicians'])
ax.legend(loc=3)
ax.set_ylabel('P(correct)')
ax.set_ylim(0, 1)
ax.set_xlim(-0.3, 1.3)
fig

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 5))

# Suj
for j, musi in enumerate(np.unique(MUSICIANS)):
    M = np.nanmean(RT_TRAINING[MUSICIANS[:, 0, 0] == musi], 1)
    m = np.nanmean(M, 0)
    se = np.nanstd(M, 0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
    label = 'Musicians' if musi == True else 'Non-musicians'
    ax.errorbar(j, m, yerr=se, marker='D', linewidth=2, markersize=7, 
                label=label, color=colors[j])

for s, suj in enumerate(sujs):
    ax.plot(np.random.normal(0, 0.01) + MUSICIANS[s, 0, 0], np.nanmean(RT_TRAINING[s], 0), marker='o', 
            linewidth=1, color=colors[MUSICIANS[s, 0, 0]], linestyle='--', markersize=4, alpha=0.3)

ax.set_xticks(np.arange(2))
ax.set_xticklabels(['Non-musicians', 'Musicians'])
ax.legend(loc=3)
ax.set_ylabel('Response time (ms)')
ax.set_xlim(-0.3, 1.3)

### Statistics.

In [ ]:
# Call R
from rpy2.robjects.packages import importr
from rpy2.robjects import IntVector, FloatVector, FactorVector, globalenv
lme4 = importr('lme4')
base = importr('base')
stats = importr('stats')

In [ ]:
_, n_training = PERF_TRAINING.shape
PERF_TRAINING.shape

In [ ]:
X = np.ravel(PERF_TRAINING)
X.shape

In [ ]:
Y1 = np.ravel(MUSICIANS[:, 0, :n_training])
Y1.shape

In [ ]:
S = np.ravel(np.transpose(np.tile(sujs, (n_training, 1)), (1, 0)))
S = np.array(S, dtype=np.float)
S.shape

In [ ]:
globalenv["music"] = FactorVector(np.array(Y1, dtype=np.str))
globalenv["perf"] = FloatVector(X)
globalenv["suj"] = FactorVector(np.array(S, dtype=np.str))

In [ ]:
m = lme4.glmer("perf ~ music + (1|suj)", family='binomial')

In [ ]:
print(base.summary(m))

# Performances musicians/non-musicians in the task.

In [ ]:
DIFF = SEQ_ID % 2 + 1

### Plot performances.

In [ ]:
PERF_DIFF = np.zeros((n_sujs, 2))
RT_DIFF = np.zeros((n_sujs, 2))
for s, suj in enumerate(sujs):
    for i, diff in enumerate([1, 2]):
        PERF_DIFF[s, i] = np.nanmean(PERF[s, DIFF[s] == diff])
        RT_DIFF[s, i] = np.nanmean(RT[s, DIFF[s] == diff])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 5), facecolor='w')

# Suj
for j, musi in enumerate(np.unique(MUSICIANS)):
    m = np.nanmean(PERF_DIFF[MUSICIANS[:, 0, 0] == musi], 0)
    se = np.nanstd(PERF_DIFF[MUSICIANS[:, 0, 0] == musi], 0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
    print(m)
    print(se)
    label = 'Musicians' if musi == True else 'Non-musicians'
    ax.errorbar([1, 2], m, yerr=se, marker='D', linewidth=2, markersize=7, 
                label=label, color=colors[j])

for s, suj in enumerate(sujs):
    ax.plot([1, 2], PERF_DIFF[s], marker='o', 
            linewidth=1, color=colors[MUSICIANS[s, 0, 0]], linestyle='--', markersize=4, alpha=0.3)

ax.set_xticks(np.arange(2)+1)
ax.set_xticklabels(['Easy', 'Hard'])
ax.axhline(1./N_VOC, color='k', linestyle='-.', alpha=0.6, linewidth=1.5)#, label='Random')
ax.legend(loc=3)
ax.set_ylabel('P(correct)')
ax.set_ylim(0.3, 0.8)
ax.set_xlim(0.7, 2.3)
fig

In [ ]:
musi = False
print(np.nanmean(PERF_DIFF[MUSICIANS[:, 0, 0] == musi, :]))
print(np.nanstd(PERF_DIFF[MUSICIANS[:, 0, 0] == musi, :])/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi)))

In [ ]:
print(np.nanmean(PERF_DIFF[:, 1]))
print(np.nanstd(PERF_DIFF[:, 1])/np.sqrt(n_sujs))

### Statistics.

In [ ]:
X = np.ravel(PERF)
X.shape

In [ ]:
Y1 = np.ravel(MUSICIANS[:, :, :n_dec])
Y2 = np.ravel(np.transpose(np.tile(DIFF, (n_dec, 1, 1)), (1, 2, 0)))
Y1.shape

In [ ]:
S = np.ravel(np.transpose(np.tile(sujs, (n_seqs, n_dec, 1)), (2, 0, 1)))
S = np.array(S, dtype=np.float)
S.shape

In [ ]:
globalenv["music"] = FactorVector(np.array(Y1 - 0.5, dtype=np.str))
globalenv["perf"] = FloatVector(X)
globalenv["suj"] = FactorVector(np.array(S, dtype=np.str))
globalenv["diff"] = FactorVector(np.array(Y2 - 1.5, dtype=np.str))

In [ ]:
m = lme4.glmer("perf ~ music * diff + (diff|suj)", family='binomial')

In [ ]:
print(base.summary(m))

### Reaction times.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 5))

# Suj
for j, musi in enumerate(np.unique(MUSICIANS)):
    m = np.nanmean(RT_DIFF[MUSICIANS[:, 0, 0] == musi], 0)
    se = np.nanstd(RT_DIFF[MUSICIANS[:, 0, 0] == musi], 0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
    label = 'Musicians' if musi == True else 'Non-musicians'
    ax.errorbar([1, 2], m, yerr=se, marker='D', linewidth=2, markersize=7, 
                label=label, color=colors[j])

for s, suj in enumerate(sujs):
    ax.plot([1, 2], RT_DIFF[s], marker='o', 
            linewidth=1, color=colors[MUSICIANS[s, 0, 0]], linestyle='--', markersize=4, alpha=0.3)

ax.set_xticks(np.arange(2)+1)
ax.set_xticklabels(['Easy', 'Hard'])
ax.legend()
ax.set_ylabel('RT (ms)')
ax.set_ylim(1000, 2500)
ax.set_xlim(0.7, 2.3)
fig

# Figure 1

In [ ]:
bad_sujs = np.array([5])

In [ ]:
W, H = np.array([10, 5])
fig = plt.figure(facecolor='w', figsize=(W, H), dpi=300)
n_col, n_row = (4, 5)
colors = ['#1f77b4', '#ff7f0e']
tmax = 60
letter_size = 15

#########################################################
###################### PARADIGM #########################
#########################################################

for i, (seq, dec, title, i_row) in enumerate(zip(
        [SEQS[0, 0], SEQS[0, 3], SEQS[0, 1], SEQS[0, 5]],
        [DEC[0, 0], DEC[0, 3], DEC[0, 1], DEC[0, 5]],
        ['Low-order statistics P(  |  )', '', 'High-order statistics P(  |     )', ''],
        [0, 1, 3, 4])):
    
    # Declare axis
    ax = plt.subplot2grid((n_row, n_col), (i_row, 0), colspan=n_col-1)
    
    # Plot
    ax.scatter(range(n_tones), seq, c=seq, s=40, cmap='Greys', vmin=0, vmax=2, linewidths=1, edgecolors='k')
    
    # Esthetics
    ax.set_xlim(-0.5, tmax - 0.5)
    ax.set_ylim(-1, 3)
    ax.set_title(title, fontweight='bold')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_yticks([])
    ax.set_xticks(np.arange(0, tmax, 5))
    if i == 0:
        ax.spines['left'].set_visible(True)
        ax.set_yticks(range(3))
        ax.set_yticklabels(['metal', 'wood', 'glass'], fontsize=8, rotation=20)
    if i in (1, 3):
        ax.set_xticklabels(np.arange(0, tmax, 5))
        ax.set_xlabel('Tone #')
    else:
        ax.set_xticklabels([''])
        
    # Decision "?"
    for _idx in np.where(dec)[0]:
        if _idx > tmax: continue
        idx = float(_idx) - 0.5#/n_tones
        p_fancy = FancyBboxPatch((idx-.9, -0.6),
                                 1.8, 3, fc=(1, 1, 1), ec='w', lw=0,
                                 boxstyle="round,pad=0.1", transform=ax.transData)
        ax.add_patch(p_fancy)
        p_fancy = FancyBboxPatch((idx-.55, 1.1),
                                 1., .2, fc='k', ec='w', lw=1,
                                 boxstyle="circle", transform=ax.transData)
        ax.add_patch(p_fancy)
        ax.text(idx-0.4, 0.8, '?', size=10, color='w', fontweight='bold', transform=ax.transData)

    if i == 0: ax.text(-0.05, 1.2, 'A', transform=ax.transAxes, fontsize=letter_size, fontweight='bold')

#########################################################
####################### RESULTS #########################
#########################################################

# Sujs
ax = plt.subplot2grid((n_row, n_col), (0, n_col-1), colspan=1, rowspan=n_row)
for j, musi in enumerate([True, False]):
    m = np.nanmean(PERF_DIFF[MUSICIANS[:, 0, 0] == musi], 0)
    se = np.nanstd(PERF_DIFF[MUSICIANS[:, 0, 0] == musi], 0)/np.sqrt(np.sum(MUSICIANS[:, 0, 0] == musi))
    label = 'Musicians' if musi == True else 'Non-musicians'
    ax.errorbar([1, 2], m, yerr=se, marker='D', linewidth=2, markersize=7, 
                label=label, color=colors[musi])

for s, suj in enumerate(sujs):
    if suj in bad_sujs: continue
    ax.plot([1, 2], PERF_DIFF[s], marker='o', 
            linewidth=1, color=colors[MUSICIANS[s, 0, 0]], linestyle='--', markersize=2, alpha=0.1)

ax.set_xticks(np.arange(2)+1)
ax.set_xticklabels(['Low-order\nstatistics', 'High-order\nstatistics'])
ax.axhline(1./N_VOC, color='k', linestyle='-.', alpha=0.6, linewidth=1.5)
ax.axhline(0.8, color='k', linestyle='-.', alpha=0.6, linewidth=1.5)
ax.legend(loc=3)
ax.set_ylabel('P(correct)')
ax.set_ylim(0.23, 0.85)
ax.set_xlim(0.7, 2.3)
ax.set_title('Performances', fontweight='bold')
ax.text(0.8, 0.81, 'optimal', fontweight='bold')
ax.text(0.8, 0.342, 'chance level', fontweight='bold')
ax.text(-0.2, 1.05, 'B', transform=ax.transAxes, fontsize=letter_size, fontweight='bold')

plt.tight_layout(h_pad=0, w_pad=3)

In [ ]:
fname = '/media/jacques/DATA1/2019_MusicPred/experimentContinuousMatrix/analysis/Figures/Figure_1.png'
fig.savefig(fname, bbox_inches='tight')